In [1]:
import numpy as np
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP
import scipy.optimize as opt
%matplotlib inline

In [2]:
#Dados do Compressor
fluid = 'Isobutane'
D_p = 22.5e-3  #Diâmetro do pistão [m]
e = 7.5e-3     #Excentricidade [m]
Cb = 38.5e-3   #Comprimento da biela [m]
dm = 2e-3      #Reversibilidade eixo-cilindro [m]
V_c = 120e-9   #Volume morto [m³]
D_os = 8e-3    #Diâmetro do orifício de sucção [m]
D_od = 6.5e-3  #Diâmetro do orifício de descarga [m]
f = 60         #Rotação [Hz]

#Dados do Ciclo
T_e = -17      #Temperatura de evaporação [oC]
#T_e = 258.15
T_c = 54.4     #Temperatura de condensação [oC]
T_s = 32.2     #Temperatura de sucção [oC]

In [3]:
def C_to_K(T): #Converte Celius para Kelvin
    return T + 273.15
def K_to_C(T): #Converte Kelvin para Celcius
    return T - 273.15

In [4]:
#Propriedades Termodinâmicas
p_e = CP.PropsSI('P','T',C_to_K(T_e),'Q',0,fluid)            #Pressão de saturação @ T_e [Pa]
p_c = CP.PropsSI('P','T',C_to_K(T_c),'Q',0,fluid)            #Pressão de saturação @ T_c [Pa]

In [46]:
#Ponto 1
p_1 = p_e
rho_sl = CP.PropsSI('D','T',C_to_K(T_s),'P',p_1,fluid)       #Densidade na linha de sucção @ T_s,p_1 [kg/m³]
s_1 = CP.PropsSI('S','T',C_to_K(T_s),'P',p_1,fluid)          #Entropia específica @ T_s,p_1 [J/kg*K]
h_1 = CP.PropsSI('H','T',C_to_K(T_s),'P',p_1,fluid)          #Entalpia específica @ T_s,p_1 [J/kg]
v_1 = 1/rho_sl                                               #Volume específico na linha de sucção @ T_s,p_1 [m³/kg]

#Ponto 2
p_2 = p_c
s_2 = s_1
h_2 = CP.PropsSI('H','S',s_2,'P',p_2,fluid)
v_2 = (CP.PropsSI('D','S',s_2,'P',p_2,fluid))**(-1)

#Ponto 3
p_3 = p_c
h_3 = CP.PropsSI('H','Q',0,'P',p_3,fluid)

#Ponto 4
p_4 = p_e
h_4 = h_3

k = np.log(p_1/p_2)/np.log(v_2/v_1)                          #Compressão adiabática e isentrópica para encontrar k = cp/cv

In [47]:
#Eficiência volumétrica
L = e*2                                                      #Stroke [m]
V_swept = np.pi*((D_p**2)/4)*L                               #Volume varrido (cilindrada) [m³]
C = V_c/V_swept                                              #Fração de volume morto [-]
eta_v = 1 + C - C*(p_c/p_e)**(1/k)                           #Eficiência volumétrica [-]

In [50]:
#Capacidade de Refrigeração
V_dot_swept = V_swept*f                                      #Vazão volumétrica [m³/s]
m_dot = eta_v * rho_sl * V_dot_swept                         #Vazão mássica [kg/s]
Q_dot_e = m_dot * (h_1 - h_4)                                #Capacidade de refrigeração [W]

#Potência Consumida
W_dot = m_dot * (h_2 - h_1)                                  #Potência consumida [W]

In [55]:
#Temperatura de Descarga
T_2K = CP.PropsSI('T','S',s_2,'P',p_2,fluid)                 #Temperatura de descarga [K]
T_2 = K_to_C(T_2K)                                           #Temperatura de descarga [C]

In [54]:
#Eficiência
COP = Q_dot_e/W_dot                                          #Coeficiente de performance [-]
COP_Carnot = C_to_K(T_e) / (C_to_K(T_c) - C_to_K(T_e))       #Coeficiente de performance de Carnot [-]
eta = COP / COP_Carnot                                       #Eficiência do cliclo [-]

In [ ]:
A = np.array([v_1,v_2,v_3,_v4])
plt.plot(A,B,"-",label="Linha")